로지스틱 회귀 모델을 파이토치 클래스로 정의합니다.

In [1]:
# 실습에 필요한 라이브러리를 불러옵니다.
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 사이킷런에서 유방암 진단 데이터 세트를 불러옵니다.
dataset = datasets.load_breast_cancer()

# 입력 데이터와 타깃을 준비합니다.
X, y = dataset['data'], dataset['target']

# 입력 데이터와 타깃을 텐서 자료구조로 변환합니다.
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

In [3]:
# 데이터 범위 조정을 위해 표준화를 적용합니다.
X = (X - torch.mean(X)) / torch.std(X)

In [4]:
# n개의 값을 입력 받는 로지스틱 회귀 모델 객체를 생성합니다.
class LogisticRegression(nn.Module):
    # 생성자에서 모델의 구조를 정의합니다. 
    def __init__(self, num_features):
        # 상속받아 생성한 객체이므로 부모(nn.Module)의 생성자를 호출합니다.
        super().__init__()
        # num_features개의 특성을 입력받는 선형 모델 객체를 생성합니다.
        self.linear = nn.Linear(num_features, 1)
        # 시그모이드 함수 객체를 생성합니다.
        self.sigmoid = nn.Sigmoid()

    # 모델의 순전파를 정의합니다.
    def forward(self, X):
        # 생성자에서 만든 선형 모델과 시그모이드 함수로 타깃을 예측하고 반환합니다.
        out = self.linear(X)
        out = self.sigmoid(out)
        return out

# 30개의 값을 입력 받는 로지스틱 회귀 모델 객체를 생성합니다.
model = LogisticRegression(30)

In [5]:
# 이진 크로스 엔트로피(Binary Cross Entropy) 손실 함수 객체를 생성합니다.
criterion = nn.BCELoss()

# Adam 옵티마이저 객체를 생성합니다.
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [6]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, X, y):
  # 기울기를 초기화합니다.
  optimizer.zero_grad()
  # 모델을 사용해 결과를 예측합니다.
  y_predicted = model(X)
  # 오차를 계산합니다.
  loss = criterion(y_predicted, y)
  # 기울기를 계산합니다.
  loss.backward()
  # 경사 하강법으로 가중치를 수정합니다.
  optimizer.step()  
  # 현재 에포크의 오차를 반환합니다.
  return loss.item()

In [7]:
# 100회에 걸쳐 모델을 학습시킵니다.
n_epochs = 100
for epoch in range(1, n_epochs+1):
  # 모델을 학습시킵니다.
  loss = train(model, criterion, optimizer, X, y)  
  # 10 epoch마다 오차를 출력합니다.
  if epoch % 10 == 0:
    print('epoch: {}, loss: {:.4f}'.format(epoch, loss))

epoch: 10, loss: 0.5904
epoch: 20, loss: 0.5220
epoch: 30, loss: 0.4726
epoch: 40, loss: 0.4356
epoch: 50, loss: 0.4069
epoch: 60, loss: 0.3841
epoch: 70, loss: 0.3655
epoch: 80, loss: 0.3501
epoch: 90, loss: 0.3371
epoch: 100, loss: 0.3261


In [8]:
# 학습된 모델을 사용해 결과를 예측합니다.
y_predicted = (model(X) >= 0.5).float()

# 정확도를 계산하고 출력합니다.
score = (y_predicted==y).float().mean()
print('accuracy: {:.2f}'.format(score))

accuracy: 0.92
